In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [26]:
#data_prefix = "ling_data/"
data_prefix = "data/"
course = pd.read_csv(data_prefix + "course.csv")
course_element = pd.read_csv(data_prefix + "course_element.csv") 
course_module = pd.read_csv(data_prefix + "course_module.csv") 
sol_log = pd.read_csv(data_prefix + "solution_log.csv") 
u_course_progress = pd.read_csv(data_prefix + "user_course_progress.csv") 
u_element_progress = pd.read_csv(data_prefix + "user_element_progress.csv") 
u_module_progress = pd.read_csv(data_prefix + "user_module_progress.csv")
users = pd.read_csv("user_ids.csv")

C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Фичи: 
course_done - получен ли сертификат 1 уровня
progress_current_ratio - доля полученных баллов за курс
progress_current - сколько всего баллов полученно 
total_courses_done - сколько всего закрыл курсов ученик
total_tries_overall - сколько попыток сдачи сделал на всех курсах 
tries_count - сколько попыток сделал ученик на данном курсе
task_problems_in_course	text_problems_in_course	video_problems_in_course - сколько в курсе соответсвующих задач
mean_progress - на сколько баллов люди в среднем прорешивают курс
total_participants - сколько всего участников было на курсе
advanced_modules_count - сколько в курсе продвинутых модулей (в геометрии 0)
transferred_count - сколько задач пользователем перенесено
tries_efficiency - кол-во баллов в курсе делить на кол-во попыток в курсе
То что еще надо юзать: 
knowledge_area_id

In [27]:
## Удалим тех, у кого не решено ни одного модуля

## ids = u_course_progress.groupby("user_id").modules_achieved.sum()
## ids = ids[ids > 0]
## u_course_progress = u_course_progress[u_course_progress.user_id.isin(ids.index)]
ids = users.user_id
u_course_progress = u_course_progress[u_course_progress.user_id.isin(ids)]

In [28]:
## Посчитаем, сколько в курсе можно заработать максимум баллов
course_to_max_points = dict(course_module.groupby("course_id").progress_max.sum())


## Подсчитаем, закрыл ли пользователь курс на сертификат первого уровня

d = dict(course_module[~course_module.is_advanced & course_module.level == 1].groupby(by="course_id").id.unique().apply(len))
u_course_progress["course_done"] = u_course_progress.course_id.map(d) <= u_course_progress.modules_achieved


## Подсчитаем по пользователю, сколько баллов он заработал

u_course_progress = u_course_progress.merge(u_element_progress.groupby(['user_id', 'course_id']).\
                                            progress_current.sum(), on=['user_id', 'course_id'])

## Найдем долю баллов которую получил пользователь от максимального числа баллов

u_course_progress["progress_current_ratio"] = u_course_progress.progress_current / u_course_progress.course_id.map(course_to_max_points)


## Найдем, сколько всего ученик закрыл курсов

u_course_progress["total_courses_done"] = u_course_progress.user_id.map(dict(u_course_progress.groupby("user_id").course_done.sum()))





## Найдем, сколько всего попыток сдачи задач, сделал ученик. total_tries_overall - считая все курсы, 
## total_tries - учитывая только данный курс

u_course_progress["total_tries_overall"] = u_course_progress.user_id.map(dict(u_element_progress.groupby("user_id").tries_count.sum()))
u_course_progress = u_course_progress.merge( u_element_progress.groupby(["user_id", "course_id"]) \
                        .tries_count.sum(), on=['user_id', 'course_id'], how = "left")





## Найдем, сколько в курсе видео / текстовых / задачный элементов

course_element["course_id"] = course_element.module_id.map(dict(zip(course_module.id, course_module.course_id)))

for name in ["task", "text", "video"]:
    u_course_progress[name + "_problems_in_course"] = u_course_progress.course_id. \
    map(dict(course_element[course_element.element_type == name].groupby("course_id").id.count()))
    
    
    


## Найдем, сколько баллов получают люди за курс в среднем

u_course_progress["mean_progress"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").progress_current_ratio.sum() / u_course_progress.groupby("course_id").progress_current_ratio.count()))


## Найдем, сколько всего участников на курсе


u_course_progress["total_participants"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").id.count()))



## Найдем, сколько продвинутых модулей на курсе
u_course_progress["advanced_modules_count"] = u_course_progress.course_id.map(course_module.groupby("course_id").is_advanced.sum())



## Посчитаем, сколько задач каждый пользователь перенес 

d = u_element_progress[u_element_progress.achieve_reason == "transferred"].groupby(["user_id", "course_id"]).id.count()
u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").id_y.fillna(0)
u_course_progress.rename(columns={'id_y':'transferred_count'}, inplace=True)


## Посчитаем кол-во баллов полученное за курс учеников разделенное на количество попыток в курсе
u_course_progress["tries_efficiency"] = u_course_progress.progress_current / u_course_progress.tries_count



u_course_progress.to_csv("user_course_progress_done.csv", index=None)


In [30]:
u_course_progress.isnull().sum()

id                            0
user_id                       0
course_id                     0
modules_achieved              0
advanced_modules_done         0
time_created                262
time_updated                537
modules_closed                0
modules_done                  0
course_done                   0
progress_current              0
progress_current_ratio        0
total_courses_done            0
total_tries_overall           0
tries_count                   0
task_problems_in_course       0
text_problems_in_course       0
video_problems_in_course      0
mean_progress                 0
total_participants            0
advanced_modules_count        0
tries_efficiency            805
dtype: int64